# Fetch Data

### Import Modules

In [ ]:
import requests
from glob import glob
from time import sleep
from tqdm.notebook import tqdm
from creds import api_creds

### Create Functions

In [ ]:
# authenticated api request
def request(endpoint, params = {}, page = 1):
    api_key = api_creds.api_key
    
    default_params = {
        'page': page,
        'page_size': 1000,
    }        
    
    return requests.get(
        f'{api_creds.base_url}/{endpoint}',
        headers = {'Authorization': api_key},
        params = { **params, **default_params }
    )

In [ ]:
# write output to JSON directory
def write(response = None, directory = "", endpoint = "", page = 1):
    with open(f'json/{directory}/{endpoint}.{page}.json', 'wb') as f:
        for chunk in response.iter_content(chunk_size=128):
            f.write(chunk)    

In [ ]:
# download full dataset from endpoint
def fetch(directory, endpoint, params = {}):
    if len(glob(f'json/{directory}/{endpoint}*')) > 0:
        print(f'Failed to Load. JSON Directory Already Contains Endpoint: {endpoint}.')
    else:
        response = request(endpoint, params = params)
        response.raise_for_status()

        write(directory = directory, response = response, endpoint = endpoint)
        
        pages = response.json()['meta']['pagination']['pages']

        for i in tqdm(range(2, pages + 1), unit = 'GET'):
            sleep(1) # limit load by waiting a second between requests
            response = request(endpoint = endpoint, params = params, page = i)
            write(response=response, directory = directory, endpoint = endpoint, page = i)     

### Test Response Code and Data Type

In [ ]:
# membership instance events
response = request('membership_instance_events')
assert response.status_code == 200
assert type(response.json()) == dict

### Execute Fetch & Write

In [ ]:
# fetch membership instance events and store in specified directory
directory = endpoint = 'membership_instance_events'
fetch(directory, endpoint)